In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
Amazon=pd.read_csv(filepath_or_buffer='amazon_india_decade_sales_data(cleaned).csv')
Amazon_unclean=pd.read_csv(filepath_or_buffer='amazon_india_complete_2015_2025.csv')

In [ ]:
Amazon


In [ ]:
Amazon_unclean

In [25]:
import pandas as pd
import numpy as np
import re

def clean_rating(value):
    """
    Standardize customer ratings to numeric (3, 3.5, 4, 4.5, 5)
    Handles: '3', '3.0', '3/5', '3.0/5.0', '3 stars', etc.
    Keeps NaN as NaN.
    """
    if pd.isna(value):
        return np.nan

    # convert to lowercase string for uniform processing
    value = str(value).strip().lower()

    # extract the first valid number from string
    match = re.search(r'\d+(\.\d+)?', value)
    if match:
        num = float(match.group())
        # convert float like 3.0 → 3, keep 3.5 as is
        return int(num) if num.is_integer() else num
    else:
        return np.nan

# ✅ apply and overwrite the column
Amazon["customer_rating"] = Amazon["customer_rating"].apply(clean_rating)


In [26]:
Amazon['customer_rating'].unique()

array([4. , 5. , 3.5, nan, 4.5, 3. ])

In [27]:
Amazon.to_csv('amazon_india_decade_sales_data(cleaned).csv', index=False)

In [4]:
Amazon.dtypes

transaction_id             object
order_date                 object
customer_id                object
product_id                 object
product_name               object
category                   object
subcategory                object
brand                      object
original_price_inr        float64
discount_percent          float64
discounted_price_inr      float64
quantity                    int64
subtotal_inr              float64
delivery_charges          float64
final_amount_inr          float64
customer_city              object
customer_state             object
customer_tier              object
customer_spending_tier     object
customer_age_group         object
payment_method             object
delivery_type              object
is_prime_member              bool
is_festival_sale             bool
festival_name              object
customer_rating           float64
return_status              object
order_month                 int64
order_year                  int64
order_quarter 

In [7]:
import mysql.connector
import pandas as pd
from sqlalchemy import create_engine
csv_path = "Amazon_india_decade_sales_data(cleaned).csv"

conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Root',
    port=3306,
    database='amazon_db'
)

dtypes = {
    "transaction_id": "string",
    "order_date": "string",
    "customer_id": "string",
    "product_id": "string",
    "product_name": "string",
    "category": "string",
    "subcategory": "string",
    "brand": "string",
    "original_price_inr": "float64",
    "discount_percent": "float64",
    "discounted_price_inr": "float64",
    "quantity": "int64",
    "subtotal_inr": "float64",
    "delivery_charges": "float64",
    "final_amount_inr": "float64",
    "customer_city": "string",
    "customer_state": "string",
    "customer_tier": "string",
    "customer_spending_tier": "string",
    "customer_age_group": "string",
    "payment_method": "string",
    "delivery_type": "string",
    "is_prime_member": "bool",
    "is_festival_sale": "bool",
    "festival_name": "string",
    "customer_rating": "float64",
    "return_status": "string",
    "order_month": "int64",
    "order_year": "int64",
    "order_quarter": "int64",
    "product_weight_kg": "float64",
    "is_prime_eligible": "bool",
    "product_rating": "float64",
    "delivery_days": "int64"
}

# --- 4️⃣ Read CSV in Chunks (efficient for >1M rows) ---
chunksize = 100000  # Adjust for your system memory

for i, chunk in enumerate(pd.read_csv(csv_path, dtype=dtypes, chunksize=chunksize)):
    print(f"🔹 Inserting chunk {i+1} ...")
    chunk.to_sql("amazon_sales", con=engine, if_exists="append", index=False)
print("✅ Data successfully loaded into SQL!")

# --- 5️⃣ Verify Data ---
with engine.connect() as conn:
    sample = pd.read_sql("SELECT * FROM amazon_sales LIMIT 5;", conn)
    print(sample)


🔹 Inserting chunk 1 ...
🔹 Inserting chunk 2 ...
🔹 Inserting chunk 3 ...
🔹 Inserting chunk 4 ...
🔹 Inserting chunk 5 ...
🔹 Inserting chunk 6 ...
🔹 Inserting chunk 7 ...
🔹 Inserting chunk 8 ...
🔹 Inserting chunk 9 ...
🔹 Inserting chunk 10 ...
🔹 Inserting chunk 11 ...
🔹 Inserting chunk 12 ...
✅ Data successfully loaded into SQL!
      transaction_id  order_date         customer_id   product_id  \
0  TXN_2023_00063013  2023-07-23  CUST_2023_00018393  PROD_000454   
1  TXN_2021_00064486  2021-07-20  CUST_2015_00002865  PROD_000579   
2  TXN_2017_00065617  2017-11-16  CUST_2016_00004057  PROD_000295   
3  TXN_2020_00054393  2020-04-05  CUST_2020_00014574  PROD_001654   
4  TXN_2018_00071646  2018-09-10  CUST_2018_00006275  PROD_000095   

                        product_name                   category  subcategory  \
0                Vivo y95 64gb black  Electronics & Accessories  Smartphones   
1        Realme realme 3 128gb black  Electronics & Accessories  Smartphones   
2                